In [30]:
from abc import ABC, abstractmethod
import pandas as pd
import datetime as dt
from hurst import compute_Hc

class Robot(ABC):
    def __init__(self,
                 username: str):
        self.username = username
        self.df = None

        self.orders = []
        self.data_from_order_book = None
        self.data_from_accountant = None
        self.new_actions = []

    def set_hist_prices(self,
                        df: 'pd.DataFrame'):
        self.df = df

    def update(self,
               data_from_order_book: dict,
               orders: list,
               data_from_accountant: dict):
        self.data_from_order_book = data_from_order_book
        self.orders = orders
        self.data_from_accountant = data_from_accountant

    def get_bid(self,
                volume_too: bool = False) -> float or ():
        if volume_too:
            return self.data_from_order_book['max_bid_price'], \
                   self.data_from_order_book['max_bid_volume']
        return self.data_from_order_book['max_bid_price']

    def get_ask(self,
                volume_too: bool = False) -> float or ():
        if volume_too:
            return self.data_from_order_book['min_ask_price'], \
                   self.data_from_order_book['min_ask_volume']
        return self.data_from_order_book['min_ask_price']

    def get_last_quote(self):
        return self.data_from_order_book['last_quote']

    def get_last_quote_time(self):
        return self.data_from_order_book['datetime']

    def get_current_balance(self):
        return self.data_from_accountant['balance']

    def get_current_net_position(self) -> (float, int):
        return self.data_from_accountant['cnp_price'], \
               self.data_from_accountant['cnp_lots']

    @staticmethod
    def format_price(price: float):
        return float(format(price, '.2f'))

    def get_active_orders(self):
        return self.orders

    def get_buy_orders(self):
        buy_orders = []
        for order in self.orders:
            if order.operation == 'buy':
                buy_orders.append(order)
        return buy_orders

    def get_sell_orders(self):
        sell_orders = []
        for order in self.orders:
            if order.operation == 'sell':
                sell_orders.append(order)
        return sell_orders

    def order_send(self,
                   operation: 'buy or sell',
                   order_type: 'limit or datasource',
                   lots: int,
                   price: float = None):
        if order_type == 'datasource':
            self.data_from_accountant['cnp_price'] = self.get_ask() if operation == 'buy' else self.get_bid()
            self.data_from_accountant['cnp_lots'] += lots
        elif order_type == 'limit':
            if operation == 'buy' and price >= self.get_ask() or operation == 'sell' and price <= self.get_bid():
                self.data_from_accountant['cnp_price'] = self.get_ask() if operation == 'buy' else self.get_bid()
                self.data_from_accountant['cnp_lots'] += lots
            else:
                self.orders.append(Order(operation,
                                   order_type,
                                   lots,
                                   self.format_price(price),
                                   self.username,
                                   self.data_from_order_book['datetime'],
                                   True,
                                   False))

    def order_delete(self,
                     order):
        for i in range(len(self.orders)):
            if self.orders[i] is order:
                self.orders.pop(i)
                return

    def order_change(self,
                     order,
                     price: float = None,
                     lots: float = None):
        order.price = price or order.price
        order.lots = lots or order.lots

    @abstractmethod
    def train(self):
        pass

    @abstractmethod
    def on_tick(self):
        pass

class RobatA(Robot):  # название класса 
    def __init__(self):
        super().__init__('RobotA') # название команды
        self.risk = 2              # риск на сделку в процентах
        self.sl = None             # стоп-лосс (с исходным значением None)
        self.tp = None             # тейк-профит (с исходным значением None)
        self.period = 20           # период скользящей средней по минутным свечкам 
        self.window = []           # список для хранеия цен закрытия свечей для подсчета среднего
        self.mean = None           # среднее (с исходным значением None) 
        self.returns = []          # список доходностей
        self.volume = 0            # объем купленного 
        self.H = 0.5               # экспонента Хёрста
        self.c = 0                 # параметр при расчете экспоненты Хёрста
        self.data = 0              # параметр при расчете экспоненты Хёрста
        self.cost = 0              # цена открытых позиций
        self.volume = 0            # объем открытых позиций
        # флаговые переменные
        self.wait_to_open_long = False  # метка открытия лонга
        self.in_long = False            # находимся ли сейчас в лонге    
      
    def train(self):
        # обратились к self.df как к обычному датафрейму и взяли последние 20 цен закрытия, потом преобразовали в список
        self.window = list(self.df['close'].iloc[-self.period:])
        self.mean = sum(self.window) / len(self.window)                        # среднее
        self.returns.append((self.df['open'] - self.df['close'])/self.df['close'])  # преобразование цен в доходности
        
    def on_tick(self):
        # получим необходимую информацию 
        # биржевые данные
        ask, ask_volume = self.get_ask(True)
        time = self.get_last_quote_time()

        # на 59 обновляем некоторых переменных        
        if time.second == 59:
            self.window.pop(0)         # удаляем первый элемент окна (по времени она будет самой далекой)
            self.window.append(price)  # добавляем в конец текущую цену (по времени она будет ближайшей)
            self.mean = sum(self.window) / len(self.window)  # пересчитываем среднее        
            self.returns.append((self.df['open'] - self.df['close'])/self.df['close'])  # пересчитаем доходности
        
        if time.second == 1:
            self.H, self.c, self.data = compute_Hc(self.returns[-21:-1], kind='change') # H, c, data - параметры библиотеки 
            if self.H > 0.5:
                if self.mean > 0:
                    self.sl = ask * 0.99  # открываемся примерно по аску, поэтому стоп ставим с отступом -1%
                    self.tp = ask * 1.02  # аналогично с тейк на +2% 
                    # определяем lots с помощью функции calc_lots
                    lots = self.calc_lots(ask)
                    self.order_send('buy', 'datasource', lots)  # выставляем ордер
                    self.wait_to_open_long = True           # обновлю флаговую переменную 
                    
            elif self.H < 0.5:
                if self.mean < 0:
                    self.sl = ask * 0.99  # открываемся примерно по аску, поэтому стоп ставим с отступом -1%
                    self.tp = ask * 1.02  # аналогично с тейк на +2% 
                    # определяем lots с помощью функции calc_lots
                    lots = self.calc_lots(ask)
                    self.order_send('buy', 'datasource', lots)  # выставляем ордер
                    self.wait_to_open_long = True           # обновлю флаговую переменную 

        elif time.second == 40:
            if self.H > 0.5:
                if self.mean < 0:
                    self.sl = ask * 0.99  # открываемся примерно по аску, поэтому стоп ставим с отступом -1%
                    self.tp = ask * 1.02  # аналогично с тейк на +2% 
                    # определяем lots с помощью функции calc_lots
                    lots = self.calc_lots(ask)
                    self.order_send('buy', 'datasource', lots)  # выставляем ордер
                    self.wait_to_open_long = True           # обновлю флаговую переменную 
                    
            elif self.H < 0.5:
                if self.mean > 0:
                    self.sl = ask * 0.99  # открываемся примерно по аску, поэтому стоп ставим с отступом -1%
                    self.tp = ask * 1.02  # аналогично с тейк на +2% 
                    # определяем lots с помощью функции calc_lots
                    lots = self.calc_lots(ask)
                    self.order_send('buy', 'datasource', lots)  # выставляем ордер
                    self.wait_to_open_long = True           # обновлю флаговую переменную 
 
        # если заявки не исполнились  к 55 секунде, снимаем их
        elif time.second == 55:
            if self.wait_to_open_long:                      # проверю флаг
                buy_orders = self.get_buy_orders()                 
                if len(buy_orders) == 0: 
                    #  исполненные ордера
                    self.wait_to_open_long = False
                    self.in_long = True
                    
                else:
                    # неисполненные ордера
                    for order in buy_orders:               # итерирация по всем ордерам 
                        for lots in order:
                            self.order_delete(order)       #  удаление ордеров
                        
        elif time.second == 58:
            self.cost, self.volume = self.get_current_position() # объем открытых позиций
            self.order_send('sale', 'datasource', volume)            # закрытие позиций
            self.volume = 0
            
    def calc_lots(self, bid):
        balance = self.get_current_balance()               # баланс
        max_loss_for_1lot = abs(self.sl - bid)             # максимальный убыток на 1 лот 
        max_available_loss = balance * self.risk / 100     # максимальный доступный по рискам убыток
        return int(max_available_loss / max_loss_for_1lot) # возвращаем максимальный лот в формате целого числа!
       